In [1]:
import os
from bifacial_radiance import *   
import pandas as pd
import irrmapping
import time

In [2]:
#%% set parameters

workfolder = os.getcwd()
if not os.path.exists(workfolder):
    os.makedirs(workfolder)
print ("Simulation will be stored in %s" % workfolder)

simulationname = 'Example2'


# Traking parameters
cumulativesky = False
limit_angle = 55 # degrees
angledelta = 0.01 #
backtrack = True

# makeModule parameters

moduletype = 'HalfCellModule'
numcellsx = 6
numcellsy = 22
xcell = 0.21
ycell = 0.105
xcellgap = 0.005
ycellgap = 0.0026
zcell = 0.0002  

x = xcell * numcellsx + xcellgap * (numcellsx-1)    
y = ycell * numcellsy + ycellgap * (numcellsy-1)    

cellModuleParams = {'numcellsx': numcellsx, 'numcellsy':numcellsy, 
                         'xcell': xcell, 'ycell': ycell, 'xcellgap': xcellgap, 'ycellgap': ycellgap}

xgap = 0.02       
ygap = 0.05         
zgap = 0.02
numpanels = 2
originx, originy, originz = (0, 0, 0)


# Scene parameters
nMods = 15
nRows = 5

albedo = 0.2
gcr = 0.4
hub_height = 3.5 # meters


CW = y * numpanels + ygap * (numpanels - 1)
pitch = CW / gcr

torquetube = True
axisofrotation = True  # the scene will rotate around the torque tube, and not the middle of the bottom surface of the module
diameter = 0.15
tubetype = 'Round'    # This will make an octagonal torque tube.
material = 'black'   # Torque tube material (0% reflectivity)
tubeParams = {'diameter':diameter,
              'tubetype':tubetype,
              'material':material,
              'axisofrotation':axisofrotation}

Simulation will be stored in C:\Users\Shuo Wang\Documents\RadianceWorkspace\radiance_work\example2


In [3]:
#%% load metdata

demo = RadianceObj(simulationname,path = workfolder)
demo.setGround(albedo)
epwfile = demo.getEPW(40.0583,-74.4057) 
starttime = '2021-06-17_0000'
endtime = '2021-06-17_1000'
metdata = demo.readWeatherFile(epwfile, starttime=starttime, endtime=endtime, coerce_year=2021) 

path = C:\Users\Shuo Wang\Documents\RadianceWorkspace\radiance_work\example2
Making path: images
Making path: objects
Making path: results
Making path: skies
Making path: EPWs
Making path: materials
Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.
Getting weather file: USA_NJ_McGuire.AFB.724096_TMY3.epw
 ... OK!
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021
Filtering dates
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos


In [4]:
#%% make scence

mymodule = ModuleObj(name=moduletype, xgap=xgap, ygap=ygap,
                zgap=zgap, numpanels=numpanels,cellModule=cellModuleParams, tubeParams=tubeParams, z=zcell)

sceneDict = {'pitch':pitch,'hub_height':hub_height, 'nMods': nMods, 'nRows': nRows}
trackerdict = demo.set1axis(limit_angle = limit_angle, backtrack = backtrack, gcr = gcr, cumulativesky = cumulativesky)
trackerdict = demo.makeScene1axis(module=moduletype, sceneDict=sceneDict) 



Module Name: HalfCellModule
Module was shifted by 0.105 in X to avoid sensors on air
This is a Cell-Level detailed module with Packaging Factor of 0.96 %
Module HalfCellModule updated in module.json

Making ~6 .rad files for gendaylit 1-axis workflow (this takes a minute..)
6 Radfiles created in /objects/


In [5]:
#%% create coordinates
# Note for tracker system, use a list to save different tilts for different timestamps
lineptsFronts = []
lineptsBacks = []
trackerkeys = sorted(demo.trackerdict.keys())
for trackerkey in trackerkeys:
    mapdict = {'xcell':xcell, 'ycell':ycell, 'zcell':zcell,
               'xcellgap':xcellgap, 'ycellgap':ycellgap, 
               'numcellsx':numcellsx, 'numcellsy':numcellsy,
               'xgap':xgap, 'ygap':ygap, 'numpanels':numpanels,
               'nMods':nMods, 'nRows':nRows, 'originx':originx, 'originy':originy,
               'height':hub_height, 'pitch':pitch,
               'offset':trackerdict[trackerkey]['scene'].module.offsetfromaxis,
               'tilt':trackerdict[trackerkey]['surf_tilt'], 
               'azimuth':trackerdict[trackerkey]['surf_azm']}
    
    lineptsFront, lineptsBack = irrmapping.genCors(**mapdict, arrayRowRange=[2], modColRange=range(3))
    
    lineptsFronts.append(lineptsFront)
    lineptsBacks.append(lineptsBack)

In [6]:
#%% Generate Skyfile & Octfiles and run simulations
irrmapping.concurMap(demo, lineptsFronts, lineptsBacks, sys='1axis', accuracy='low')


Creating ~6 skyfiles. 
Created 6 skyfiles in /skies/
Created Example2_1axis_2021-06-17_0500.oct
Created Example2_1axis_2021-06-17_1000.oct
Created Example2_1axis_2021-06-17_0600.oct
Created Example2_1axis_2021-06-17_0700.oct
Created Example2_1axis_2021-06-17_0800.oct
Created Example2_1axis_2021-06-17_0900.oct
Full scan in process for: Example2_1axis_2021-06-17_0500_Front
Full scan in process for: Example2_1axis_2021-06-17_0600_Front
Full scan in process for: Example2_1axis_2021-06-17_0700_Front
Full scan in process for: Example2_1axis_2021-06-17_0800_Front
Full scan in process for: Example2_1axis_2021-06-17_0900_Front
Full scan in process for: Example2_1axis_2021-06-17_1000_Front
Full scan in process for: Example2_1axis_2021-06-17_0500_Back
Full scan in process for: Example2_1axis_2021-06-17_0600_Back
Full scan in process for: Example2_1axis_2021-06-17_0900_Back
Full scan in process for: Example2_1axis_2021-06-17_1000_Back
Full scan in process for: Example2_1axis_2021-06-17_0700_Back
F